## Emoji prediction based on text


In [1]:
import numpy as np
import pandas as pd
import emoji

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

C:\Users\Abdulmunim\AppData\Local\Temp\ipykernel_12092\2654145384.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('data/emoji_data.csv', header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [3]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}


def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [4]:
X = data[0].values
y = data[1].values

## Embeddings


In [5]:
# I will use 50d pretrained embeddings
content = []
with open("pretrained/glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    content = f.readlines()
content[:5]

['the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581\n',
 ', 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392\n',
 '. 0.15164 0.30177 -0.16763 0.17684 0.31719 0.33973 -0.43478 -0.31086 -0.44999 -0.29486 0.16608 0.11963 -0.

In [6]:
embeddings = {}

for line in content:
    line = line.split()
    word, vector = line[0], np.array(line[1:], dtype=float)
    embeddings[word] = vector

In [7]:
def get_maxlen(X):
    maxlen = 0
    for sentence in X:
        maxlen = max(maxlen, len(sentence))
    return maxlen

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [9]:
X_seq = tokenizer.texts_to_sequences(X)
X_seq[:5]

[[103, 104, 3, 6, 105], [106, 3, 107], [1, 7, 108], [109, 4, 35], [36, 30]]

In [10]:
maxlen = get_maxlen(X_seq)
X_seq_padded = pad_sequences(
    X_seq, maxlen=maxlen, padding='post', truncating='post')

In [11]:
y_train = to_categorical(y)

## Model


In [12]:
embed_size = 50  # we used the 50d embeddings
embedding_matrix = np.zeros((len(word2index) + 1, embed_size))

for word, i in word2index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_matrix.shape

(313, 50)

In [27]:

model = Sequential(
    [
        Embedding(input_dim=len(word2index) + 1,
                  output_dim=embed_size,
                  input_length=maxlen,
                  weights=[embedding_matrix],
                  trainable=False
                  ),
        LSTM(units=16, return_sequences=True),
        LSTM(units=4),
        Dense(5, activation='softmax')
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics='accuracy')

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 10, 50)            15650     
                                                                 
 lstm_3 (LSTM)               (None, 10, 16)            4288      
                                                                 
 lstm_4 (LSTM)               (None, 4)                 336       
                                                                 
 dense (Dense)               (None, 5)                 25        
                                                                 
Total params: 20299 (79.29 KB)
Trainable params: 4649 (18.16 KB)
Non-trainable params: 15650 (61.13 KB)
_________________________________________________________________


In [30]:
model.fit(X_seq_padded, y_train, epochs=100,
          batch_size=32, validation_split=0.1)

Epoch 1/100
6/6 [==============================] - 0s 37ms/step - loss: 0.6035 - accuracy: 0.8780 - val_loss: 1.2940 - val_accuracy: 0.4211
Epoch 2/100
6/6 [==============================] - 0s 25ms/step - loss: 0.6090 - accuracy: 0.8780 - val_loss: 1.2789 - val_accuracy: 0.5789
Epoch 3/100
6/6 [==============================] - 0s 25ms/step - loss: 0.6063 - accuracy: 0.8902 - val_loss: 1.3101 - val_accuracy: 0.6842
Epoch 4/100
6/6 [==============================] - 0s 23ms/step - loss: 0.5928 - accuracy: 0.8902 - val_loss: 1.2361 - val_accuracy: 0.5789
Epoch 5/100
6/6 [==============================] - 0s 27ms/step - loss: 0.5952 - accuracy: 0.8902 - val_loss: 1.2105 - val_accuracy: 0.5789
Epoch 6/100
6/6 [==============================] - 0s 24ms/step - loss: 0.5712 - accuracy: 0.9024 - val_loss: 1.2239 - val_accuracy: 0.6316
Epoch 7/100
6/6 [==============================] - 0s 26ms/step - loss: 0.5404 - accuracy: 0.9207 - val_loss: 1.2214 - val_accuracy: 0.5263
Epoch 8/100
6/6 [===

In [31]:
test = ["I love you", "I feel good", "I am hungry",
        "I am tired", "Lets eat together"]
test_seq = tokenizer.texts_to_sequences(test)
test_seq_padded = pad_sequences(
    test_seq, maxlen=maxlen, padding='post', truncating='post')

In [34]:
y_pred = model.predict(test_seq_padded).argmax(axis=1)

1/1 [==============================] - 0s 39ms/step


In [35]:
for i in range(len(test)):
    print(f"{test[i]} - {label_to_emoji(y_pred[i])}")

I love you - ❤️
I feel good - 😃
I am hungry - 🍽️
I am tired - 😃
Lets eat together - 🍽️
